# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [6]:
import pandas as pd
import numpy as np
import re

url_airbnb = "https://raw.githubusercontent.com/KendallFreese/wrangling_assignment/main/data/airbnb_hw.csv"
air = pd.read_csv(url_airbnb)

url_mn = "https://raw.githubusercontent.com/KendallFreese/wrangling_assignment/main/data/mn_police_use_of_force.csv"
mn = pd.read_csv(url_mn)

url_justice = "https://raw.githubusercontent.com/KendallFreese/wrangling_assignment/main/data/justice_data.parquet"
justice = pd.read_parquet(url_justice)

## Q1.1

air.head()
# Keep original for comparison
air["_Price_raw"] = air["Price"].astype(str)


# Strip currency symbols, commas, spaces, and any trailing text; coerce bad parses to NaN
air["Price_clean"] = (
    air["Price"]
      .astype(str)
      .str.replace(r"[^0-9\.\-]", "", regex=True)   # remove $ , spaces, etc.
      .str.replace(r"(?<=\d)\.(?=\d{3}\b)", "", regex=True)  # handle thousands dots if present
)

# Convert to numeric
air["Price_clean"] = pd.to_numeric(air["Price_clean"], errors="coerce")

print("Q1.1 — Price cleaning")
print(air.loc[air["_Price_raw"].ne(air["Price_clean"].astype(str)) , ["_Price_raw","Price_clean"]].head(8))
print(f"\nTotal rows: {len(air):,}")
print(f"Missing after cleaning (Price_clean is NaN): {air['Price_clean'].isna().sum():,}")

# The choices I made to clean the Price variable include removing potential $ signs, spaces, trailing text, and
# commas. I also coerced missing values to read as NaN and any non-numeric strings "-" or "N/A". Therefore,
# there are no missing values after coersion as they now read as "NaN".
# Values like '1,112' become 1112 because we remove the comma before casting to numeric.


    Host Id Host Since                                Name Neighbourhood   \
0   5162530        NaN     1 Bedroom in Prime Williamsburg       Brooklyn   
1  33134899        NaN     Sunny, Private room in Bushwick       Brooklyn   
2  39608626        NaN                Sunny Room in Harlem      Manhattan   
3       500  6/26/2008  Gorgeous 1 BR with Private Balcony      Manhattan   
4       500  6/26/2008            Trendy Times Square Loft      Manhattan   

  Property Type  Review Scores Rating (bin)        Room Type  Zipcode  Beds  \
0     Apartment                         NaN  Entire home/apt  11249.0   1.0   
1     Apartment                         NaN     Private room  11206.0   1.0   
2     Apartment                         NaN     Private room  10032.0   1.0   
3     Apartment                         NaN  Entire home/apt  10024.0   3.0   
4     Apartment                        95.0     Private room  10036.0   3.0   

   Number of Records  Number Of Reviews Price  Review Scores R

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)